# Budget Analysis of 2D Turbulence

In [4]:
# Energy Equation

import numpy as np
import h5py

Re = 20000
fkx = 4
fky = 0
chi = 0.1
beta = 0

dealias= False


dataType = 'Re20kNX1024nx4ny0r0p1' # 'Re20kNX1024nx25ny25r0p1'
# filterType = 'gaussian'
# Ngrid = 32
# DATA_DIR = '../../data/2D_FHIT/' + dataType + '/' + filterType + '/NX' + str(Ngrid) + '/'


# with h5py.File(DATA_DIR + 'PsiVor.mat', 'r') as f:
#     Omega_arr = f['Vor'][:].T

# with h5py.File(DATA_DIR + 'S.mat', 'r') as f:
#     Tau11_arr = f['S1'][:].T
#     Tau12_arr = f['S2'][:].T
#     Tau22_arr = f['S3'][:].T

# with h5py.File(DATA_DIR + 'Svor.mat', 'r') as f:
#     sigma1_arr = f['Svor1'][:].T
#     sigma2_arr = f['Svor2'][:].T

#*******************************************************************************

DATA_DIR_DNS = '../../data/2D_FHIT/' + dataType + '/DNS/train1/'
with h5py.File(DATA_DIR_DNS + 'DNS1.mat', 'r') as f:
    Omega_arr = f['slnWorDNS'][:].T

Tau11_arr = np.zeros(Omega_arr.shape)
Tau12_arr = np.zeros(Omega_arr.shape)
Tau22_arr = np.zeros(Omega_arr.shape)

sigma1_arr = np.zeros(Omega_arr.shape)
sigma2_arr = np.zeros(Omega_arr.shape)

Ngrid = 1024

countSnap = 0


In [5]:
from py2d.convert import Omega2Psi, Psi2UV, strain_rate
from py2d.derivative import derivative
from py2d.initialize import initialize_wavenumbers_rfft2, gridgen
from py2d.dealias import multiply_dealias
from py2d.filter import filter2D

def energy_budget(Re, fkx, fky, chi, beta, Omega, Tau11, Tau12, Tau22, dealias=False):
    Lx, Ly = 2*np.pi, 2*np.pi
    Kx, Ky, Kabs, Ksq, invKsq = initialize_wavenumbers_rfft2(Ngrid, Ngrid, Lx, Ly)
    Lx, Ly, X, Y, dx, dy = gridgen(Lx, Ly, Ngrid, Ngrid)

    Psi = Omega2Psi(Omega, invKsq)
    U, V = Psi2UV(Psi, Kx, Ky)
    S11, S12, S22 = strain_rate(Psi, Kx, Ky)

    # Advection of Energy
    E = 0.5 * (multiply_dealias(U,U, dealias=dealias) + multiply_dealias(V,V, dealias=dealias))
    Ex = derivative(E, [1,0], Kx, Ky)
    Ey = derivative(E, [0,1], Kx, Ky)

    E_advection = U*Ex + V*Ey

    # Divergence of u_i S_ij
    u_S11 = multiply_dealias(U, S11, dealias=dealias)
    v_S12 = multiply_dealias(V, S12, dealias=dealias)
    u_S12 = multiply_dealias(U, S12, dealias=dealias)
    v_S22 = multiply_dealias(V, S22, dealias=dealias)

    u_S_divergence = derivative(u_S11 + v_S12, [1,0], Kx, Ky) + derivative(u_S12 + v_S22, [0,1], Kx, Ky)

    # Divergence of u_i Tau_ij
    u_Tau11 = multiply_dealias(U, Tau11, dealias=dealias)
    v_Tau12 = multiply_dealias(V, Tau12, dealias=dealias)
    u_Tau12 = multiply_dealias(U, Tau12, dealias=dealias)
    v_Tau22 = multiply_dealias(V, Tau22, dealias=dealias)

    u_Tau_divergence = derivative(u_Tau11 + v_Tau12, [1,0], Kx, Ky) + derivative(u_Tau12 + v_Tau22, [0,1], Kx, Ky)

    # Energy dissipation
    S11_S11 = multiply_dealias(S11,S11, dealias=dealias)
    S12_S12 = multiply_dealias(S12,S12, dealias=dealias)
    S22_S22 = multiply_dealias(S22,S22, dealias=dealias)
    E_dissipation = 2/Re * (S11_S11 + 2*S12_S12 + S22_S22)

    # Energy production
    Tau11_S11 = multiply_dealias(Tau11,S11, dealias=dealias)
    Tau12_S12 = multiply_dealias(Tau12,S12, dealias=dealias)
    Tau22_S22 = multiply_dealias(Tau22,S22, dealias=dealias)
    P_Tau = - (Tau11_S11 + 2*Tau12_S12 + Tau22_S22)

    # Energy Production from Forcing
    F1 = np.sin(fky*Y) 
    F2 = -np.sin(fkx*X)
    # F1_f = filter2D(F1, filterType=filterType, coarseGrainType=None, Delta=2*Lx/Ngrid)
    # F2_f = filter2D(F2, filterType=filterType, coarseGrainType=None, Delta=2*Ly/Ngrid)
    F1_f = F1
    F2_f = F2

    u_F1_f = multiply_dealias(U, F1_f, dealias=dealias)
    v_F2_f = multiply_dealias(V, F2_f, dealias=dealias)

    F_production = u_F1_f + v_F2_f

    # Energy decay due to Rayleigh drag
    R_decay = - 2*chi * E

    E_total =  - E_dissipation - P_Tau + F_production + R_decay

    return np.mean(E_advection), np.mean(u_S_divergence), np.mean(u_Tau_divergence), np.mean(E_dissipation), np.mean(P_Tau), np.mean(F_production), np.mean(R_decay), np.mean(E_total)


def enstrophy_budget(Re, fkx, fky, chi, beta, Omega, Sigma1, Sigma2, dealias=False):

    Lx, Ly = 2*np.pi, 2*np.pi
    Kx, Ky, Kabs, Ksq, invKsq = initialize_wavenumbers_rfft2(Ngrid, Ngrid, Lx, Ly)
    Lx, Ly, X, Y, dx, dy = gridgen(Lx, Ly, Ngrid, Ngrid)

    Psi = Omega2Psi(Omega, invKsq)
    U,V = Psi2UV(Psi, Kx, Ky)

    # Advection of enstrophy
    Z = 0.5 * multiply_dealias(Omega, Omega, dealias=dealias)
    Zx = derivative(Z, [1,0], Kx, Ky)
    Zy = derivative(Z, [0,1], Kx, Ky)

    Z_advection = multiply_dealias(U,Zx, dealias=dealias) + multiply_dealias(V,Zy, dealias=dealias)

    # Divergence Omega (Grad Omega)
    Omega_x = derivative(Omega, [1,0], Kx, Ky)
    Omega_y = derivative(Omega, [0,1], Kx, Ky)
    Omega_Omega_x = multiply_dealias(Omega, Omega_x, dealias=dealias)
    Omega_Omega_y = multiply_dealias(Omega, Omega_y, dealias=dealias)

    Omega_divergence = 1/Re * derivative(Omega_Omega_x, [1,0], Kx, Ky) + derivative(Omega_Omega_y, [0,1], Kx, Ky)

    # Divergence of Omega Sigma_i
    Omega_Sigma1 = multiply_dealias(Omega, Sigma1, dealias=dealias)
    Omega_Sigma2 = multiply_dealias(Omega, Sigma2, dealias=dealias)
    Omega_Sigma_divergence = derivative(Omega_Sigma1, [1,0], Kx, Ky) + derivative(Omega_Sigma2, [0,1], Kx, Ky)

    # Enstrophy dissipation
    Omega_x_Omega_x = multiply_dealias(Omega_x, Omega_x, dealias=dealias)
    Omega_y_Omega_y = multiply_dealias(Omega_x, Omega_y, dealias=dealias)

    eta = 1/Re * (Omega_x_Omega_x + Omega_y_Omega_y)

    # Enstrophy production
    Pz =  multiply_dealias(Sigma1, Omega_x, dealias=dealias) + multiply_dealias(Sigma2, Omega_y, dealias=dealias)

    # Enstrophy production from forcing
    f = fkx*np.cos(fkx*X) + fky*np.cos(fky*Y)
    F_production_ens = - multiply_dealias(f, Omega, dealias=dealias)

    # Enstrophy decay due to Rayleigh drag
    R_decay = - 2*chi * Z

    Z_total = - eta - Pz + F_production_ens + R_decay

    return np.mean(Z_advection), np.mean(Omega_divergence), np.mean(Omega_Sigma_divergence), np.mean(eta), np.mean(Pz), np.mean(F_production_ens), np.mean(R_decay), np.mean(Z_total)




In [6]:
E_advection_arr = []
u_S_divergence_arr = []
u_Tau_divergence_arr = []
E_dissipation_arr = []
P_Tau_arr = []
F_production_arr = []
R_decay_arr = []
E_total_arr = []

Z_advection_arr = []
Omega_divergence_arr = []
Omega_Sigma_divergence_arr = []
eta_arr = []
Pz_arr = []
F_production_ens_arr = []
R_decay_arr = []
Z_total_arr = []

for countSnap in range(20):
    Omega = Omega_arr[:,:,countSnap]
    Tau11 = Tau11_arr[:,:,countSnap]
    Tau12 = Tau12_arr[:,:,countSnap]
    Tau22 = Tau22_arr[:,:,countSnap]
    Sigma1 = sigma1_arr[:,:,countSnap]
    Sigma2 = sigma2_arr[:,:,countSnap]

    E_advection, u_S_divergence, u_Tau_divergence, E_dissipation, P_Tau, F_production, R_decay, E_total = energy_budget(Re, fkx, fky, chi, beta, Omega, Tau11, Tau12, Tau22, dealias=dealias)

    Z_advection, Omega_divergence, Omega_Sigma_divergence, eta, Pz, F_production_ens, R_decay, Z_total = enstrophy_budget(Re, fkx, fky, chi, beta, Omega, Sigma1, Sigma2, dealias=dealias)

    E_advection_arr.append(E_advection)
    u_S_divergence_arr.append(u_S_divergence)
    u_Tau_divergence_arr.append(u_Tau_divergence)
    E_dissipation_arr.append(E_dissipation)
    P_Tau_arr.append(P_Tau)
    F_production_arr.append(F_production)
    R_decay_arr.append(R_decay)
    E_total_arr.append(E_total)

    Z_advection_arr.append(Z_advection)
    Omega_divergence_arr.append(Omega_divergence)
    Omega_Sigma_divergence_arr.append(Omega_Sigma_divergence)
    eta_arr.append(eta)
    Pz_arr.append(Pz)
    F_production_ens_arr.append(F_production_ens)
    R_decay_arr.append(R_decay)
    Z_total_arr.append(Z_total)

print('****************** Energy Budget ******************')
print('E_advection = ', np.mean(E_advection_arr))
print('u_S_divergence = ', np.mean(u_S_divergence_arr))
print('u_Tau_divergence = ', np.mean(u_Tau_divergence_arr))
print('E_dissipation = ', np.mean(E_dissipation_arr))
print('P_Tau = ', np.mean(P_Tau_arr))
print('F_production = ', np.mean(F_production_arr))
print('R_decay = ', np.mean(R_decay_arr))
print('Total = ', np.mean(E_total_arr))

# total =  -np.asarray(E_dissipation_arr) - np.asarray(P_Tau_arr) + np.asarray(F_production_arr) + np.asarray(R_decay_arr)
# print('Total = ', np.mean(total))

print('****************** Enstrophy Budget ******************')
print('Z_advection = ', np.mean(Z_advection_arr))
print('Omega_divergence = ', np.mean(Omega_divergence_arr))
print('Omega_Sigma_divergence = ', np.mean(Omega_Sigma_divergence_arr))
print('eta = ', np.mean(eta_arr))
print('Pz = ', np.mean(Pz_arr))
print('F_production = ', np.mean(F_production_ens_arr))
print('R_decay = ', np.mean(R_decay_arr))
print('Total = ', np.mean(Z_total_arr))

# total =  -np.asarray(eta_arr) - np.asarray(Pz_arr) + np.asarray(F_production_ens_arr) + np.asarray(R_decay_arr)
# print('Total = ', np.mean(total))

****************** Energy Budget ******************
E_advection =  5.5619571448506374e-18
u_S_divergence =  -2.0643209364124005e-17
u_Tau_divergence =  0.0
E_dissipation =  0.001266188265777029
P_Tau =  0.0
F_production =  0.20850441987235713
R_decay =  -2.532376561466358
Total =  -0.0010386264750241818
****************** Enstrophy Budget ******************
Z_advection =  2.2542731570318606e-16
Omega_divergence =  -9.544719615367515e-16
Omega_Sigma_divergence =  0.0
eta =  0.40985242327569765
Pz =  0.0
F_production =  3.336070717957713
R_decay =  -2.532376561466358
Total =  0.3938417332156582
